# Data Scaling and Modeling

After taking a closer look at the targets (see my [first notebook](https://www.kaggle.com/miykael/trends-exploration-of-the-targets)) and performing feature exploration and engineering (see my [second notebook](https://www.kaggle.com/miykael/trends-feature-exploration-engineering)), we are ready to train some models and perform some predictions.

With this notebook I want to show how I went from an average performing model to one in the top 25th, just by **(1)** using adapted targets, **(2)** use engineered features from the MRI maps and **(3)** fine tune the scaling of the different datasets. Especially the last point is something that went against my initial intuition.

In [ ]:
import numpy as np
import pandas as pd
from os.path import join as opj
from tqdm.notebook import tqdm

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.set_context('notebook')

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.svm import SVR

# 1. Load targets and features

First things first, let's load the adapted targets and prepared feature datasets from my [second notebook](https://www.kaggle.com/miykael/trends-feature-exploration-engineering).

In [ ]:
path = '/kaggle/input/trends-feature-exploration-engineering/datasets'

In [ ]:
# Load target values and corresponding scaler
import joblib
scaler_targets = joblib.load(opj(path, 'targets_scaler.pkl'))

targets = pd.read_hdf(opj(path, 'targets.h5'))
targets.head()

To load the feature matrix, let's write a function that performs the following steps:

1. Load the train and test set.
2. Select the target feature from the 5 (resp. 7) targets.
3. Centralize the corr_coef features to the median, which is about 0.689
4. Scale the four feature datasets (IC, FNC, intra and inter correlations) according to a scaler called `scale_values`. Before this scaler is applied, all feature within a dataset are scaled to the 90% value (i.e. top 10%). No centralization was applied.
5. Missing values are dropped.

In [ ]:
def load_dataset_and_scale(target, dataset_id='merge', scale_values=[1, 1, 1, 1]):

    # Load dataset
    X_tr = pd.read_hdf(opj(path, '%s_train.h5' % dataset_id))
    X_te = pd.read_hdf(opj(path, '%s_test.h5' % dataset_id))

    # Specify target
    y = pd.read_hdf(opj(path, 'targets.h5'))
    y_tr = y.loc[X_tr.index, target]

    # Remove missing values
    missval = y_tr.isnull().values
    idx = ~missval
    X_tr = X_tr[idx]
    X_te = X_te
    y_tr = y_tr[idx]
    print('Removing %s missing values from target dataset.' % missval.sum())
    
    # Centralize corr_coef features
    median_offset = X_tr.iloc[:, X_tr.columns.str.contains('corr_coef')].median().mean()
    X_tr.iloc[:, X_tr.columns.str.contains('corr_coef')] -= median_offset
    X_te.iloc[:, X_te.columns.str.contains('corr_coef')] -= median_offset

    # Establish masks for different kinds of data
    mask_ids = []
    for m in ['IC_', '_vs_', 'corr_coef', '^c[0-9]+_c[0-9]+']:
        mask_ids.append(X_tr.columns.str.contains(m))
    mask_ids = np.array(mask_ids)

    # Data scaling
    for i, m in enumerate(mask_ids):

        if m.sum()==0:
            continue
        
        # Apply Scale
        scale_value = scale_values[i]
        unify_mask_scale = np.percentile(X_tr.iloc[:, m].abs(), 90)

        X_te.iloc[:, m] /= unify_mask_scale
        X_tr.iloc[:, m] /= unify_mask_scale
    
        X_te.iloc[:, m] *= scale_value
        X_tr.iloc[:, m] *= scale_value

    # Drop irrelevant measurements
    X_tr.dropna(axis=1, inplace=True)
    X_te.dropna(axis=1, inplace=True)
    
    # Drop duplicate rows
    X_tr = X_tr.T.drop_duplicates().T
    X_te = X_te.T.drop_duplicates().T
    
    print('Size of dataset (train/test): ', X_tr.shape, X_te.shape)
    
    X_tr = X_tr.values
    X_te = X_te.values
    y_tr = y_tr.values
    
    return X_tr, X_te, y_tr

# 2. Define scoring function and model parameters

Now that the data is ready, let's write a scoring function which **(1)** reverts the target adaptations (scaling and power transformation) and **(2)** makes sure that the predicted values are within the scope of the target values. The actual scoring function is the mean absolute error, as defined by the competition.

In [ ]:
# Create scorer function
from sklearn.metrics import make_scorer
def model_metric(y_true, y_pred, scaler=None, tidx=0):
    
    # List of power transformations
    pow_age = 1.0
    pow_d1v1 = 1.5
    pow_d1v2 = 1.5
    pow_d2v1 = 1.5
    pow_d2v2 = 1.5
    pow_d21 = 1.5
    pow_d22 = 1.0

    powers = [pow_age, pow_d1v1, pow_d1v2, pow_d2v1, pow_d2v2, pow_d21, pow_d22]
    
    # Invert scaler
    t_true = scaler.inverse_transform(np.transpose([y_true] * 7))[:, tidx]
    t_pred = scaler.inverse_transform(np.transpose([y_pred] * 7))[:, tidx]
    
    # Assign closest value from training set
    unique_values = np.unique(t_true)
    for i, a in enumerate(t_pred):
        t_pred[i] = unique_values[np.argmin(np.abs(a-unique_values))]

    # Invert power transformation
    t_true = np.power(t_true, 1./powers[tidx])
    t_pred = np.power(t_pred, 1./powers[tidx])
    
    # Compute the score
    score = np.mean(np.sum(np.abs(t_true - t_pred), axis=0) / np.sum(t_true, axis=0))
    return score

Let's also write a function that generates the modeling pipeline, parameter grid and runs the `GridSearchCV` object.

In [ ]:
def create_grid(model_metric, alphas=[0.1, 1, 10], estimator=None,
                cv=5, scaler_targets=None, tidx=0):

    # Create Pipeline
    pipeline = Pipeline([
        ('scaler', None),
        ('estimator', estimator),
    ])

    # Define parameter grid
    param_grid = [{'scaler': [None, RobustScaler()],
                   'estimator__alpha': alphas,
                  }]

    # Create grid search object
    f_scorer = make_scorer(model_metric, greater_is_better=False,
                           scaler=scaler_targets, tidx=tidx)
    grid = GridSearchCV(pipeline,
                        cv=cv,
                        param_grid=param_grid,
                        scoring=f_scorer,
                        return_train_score=True,
                        verbose=5,
                        n_jobs=-1)

    return grid

# 3. Initial data modeling approach (standard)

My first data modeling approach was rather standard. Take the data, apply a scaler, run a Ridge regression. So let's try this out, first only on the IC and FNC features and than once also with the feature engineered features.

## 3.1. Data modeling with 2 feature sets and RobustScaler

In [ ]:
# Select the target (name and index)
target = 'age'
tidx = 0

# Let's select which features to use and how to scale them
scale_values = [1,        # IC features
                1,        # FNC features
                np.nan,   # intra features
                np.nan]   # inter features

X_tr, X_te, y_tr =  load_dataset_and_scale(target, scale_values=scale_values)

In [ ]:
# Define estimator
estimator = Ridge(tol=1e-3)

# Create grid search object
alphas = np.logspace(0, 4, 21)
grid = create_grid(model_metric, alphas=alphas, estimator=estimator,
                   cv=5, scaler_targets=scaler_targets, tidx=tidx)

# Run grid search
_ = grid.fit(X_tr, y_tr)

# Provide some insights into the models top performance
print('Dataset scales used: ', scale_values)
print("Best score at: %f using %s" % (grid.best_score_, grid.best_params_))

Now to better understand what exactly happend, lets write some helper functions.

### Extract predictions and plot them

In [ ]:
def extract_predictions(X_tr, X_te, grid, y_tr):

    # Store predictions in dictionary
    res = {}
    res['tr'] = grid.predict(X_tr)
    res['te'] = grid.predict(X_te)
    
    # Assign closest value from training set 
    unique_values = np.unique(y_tr)
    for t in ['tr', 'te']:
        for i, a in enumerate(res[t]):
            res[t][i] = unique_values[np.argmin(np.abs(a-unique_values))]

    return res['tr'], res['te']

In [ ]:
# Extract the predictions for the training and the test set
pred_tr, pred_te = extract_predictions(X_tr, X_te, grid, y_tr)

In [ ]:
def plot_predictions(pred_tr, pred_te, y_tr):

    # Plot prediction descrepancy on training and test set
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

    ax[0].set_title('Prediction X_te: %s' % target)
    ax[0].plot(pred_te, '.', alpha=0.5, markersize=5)
    ax[0].plot(pred_tr, '.', alpha=0.3, markersize=5)
    ax[0].legend(['Train', 'Test'])
    ax[0].set_ylabel('target value')
    ax[0].set_xlabel('Sample')

    ax[1].set_title('Prediction X_tr: %s' % target)
    sns.regplot(x=pred_tr, y=y_tr, marker='.', ax=ax[1], scatter_kws={'s':10})
    ax[1].set_xlim([-3.5, 3.5])
    ax[1].set_ylim([-3.5, 3.5])
    ax[1].set_ylabel('True value')
    ax[1].set_xlabel('Predicted value')
    
    plt.show()

In [ ]:
# Plot the predictions with respect to each other and to the target
plot_predictions(pred_tr, pred_te, y_tr)

The **left** figure shows the individual prediction values (in y direction) in a sequential fashion next to each other (i.e. feature 0 to 5835 = `X_tr.shape[1]`). In blue are the values from the training set, in orange the one from the test set.

In the **right** figure, we can see the relationship between the predicted and the true target values.

### Collect information about grid point performance and plot them

In [ ]:
def create_df_pred(grid):

    # Store grid search parameters and outcomes in dataframe
    df_pred = pd.DataFrame(grid.cv_results_)
    columns = [c for c in df_pred.columns if 'time' not in c
               and 'split' not in c
               and 'rank' not in c
               and c!='params']
    df_pred = df_pred[columns].sort_values('mean_test_score', ascending=False)
    df_pred['param_estimator__alpha'] = df_pred['param_estimator__alpha'].astype('float')
    df_pred['param_scaler'] = df_pred['param_scaler'].astype('str')
    
    return df_pred

In [ ]:
# Creates dataframe about grid point's performance
df_pred = create_df_pred(grid).sort_values('mean_test_score')
df_pred.head()

In [ ]:
def plot_hyperparam_fitting(df_pred):

    # Plot the model fit information
    for s in df_pred['param_scaler'].unique():

        df_plot = df_pred[np.prod([df_pred['param_scaler']==s],
                                  axis=0).astype('bool')]

        df_plot = df_plot.sort_values('param_estimator__alpha')

        # Extract relevant modelling metrics
        train_scores = df_plot['mean_train_score']
        valid_scores = df_plot['mean_test_score']
        std_tr = df_plot['std_train_score']
        std_va = df_plot['std_test_score']

        plt.figure(figsize=(12, 4))
        alphas = df_plot['param_estimator__alpha']
        plt.semilogx(alphas, train_scores, label='Training Set')
        plt.semilogx(alphas, valid_scores, label='Validation Set')

        # Add marker and text for best score
        max_id = np.argmax(valid_scores)
        x_pos = alphas.iloc[max_id]
        y_pos = valid_scores.iloc[max_id]
        txt = '{:0.4f}'.format(y_pos)
        plt.scatter(x_pos, y_pos, marker='x', c='red', zorder=10)
        plt.text(x_pos, y_pos, txt, fontdict={'size': 18})

        # Quantify variance with ±std curves
        plt.fill_between(alphas, train_scores-std_tr, train_scores+std_tr, alpha=0.3)
        plt.fill_between(alphas, valid_scores-std_va, valid_scores+std_va, alpha=0.3)
        plt.ylabel('Performance metric')
        plt.xlabel('Model parameter')

        # Adjust x-lim, y-lim, add legend and adjust layout
        plt.legend()
        plt.title('Scaler: %s' % s)
        plt.show()

In [ ]:
# Plot prediction behaviour
plot_hyperparam_fitting(df_pred)

## Observation 1

As we can see, the unviersally scaled (i.e. using `RobustScaler()`) data leads to better predictions. Let's see what happens if we include all 4 feature sets.

## 3.2. Data modeling with 4 feature sets and RobustScaler

In [ ]:
# Select the target (name and index)
target = 'age'
tidx = 0

# Let's select which features to use and how to scale them
scale_values = [1,        # IC features
                1,        # FNC features
                1,        # intra features
                1]        # inter features

X_tr, X_te, y_tr =  load_dataset_and_scale(target, scale_values=scale_values)

In [ ]:
# Define estimator
estimator = Ridge(tol=1e-3)

# Create grid search object
alphas = np.logspace(1, 5, 21)
grid = create_grid(model_metric, alphas=alphas, estimator=estimator,
                   cv=5, scaler_targets=scaler_targets, tidx=tidx)

# Run grid search
_ = grid.fit(X_tr, y_tr)

# Provide some insights into the models top performance
print('Dataset scales used: ', scale_values)
print("Best score at: %f using %s" % (grid.best_score_, grid.best_params_))

In [ ]:
# Extract the predictions for the training and the test set
pred_tr, pred_te = extract_predictions(X_tr, X_te, grid, y_tr)

In [ ]:
# Plot the predictions with respect to each other and to the target
plot_predictions(pred_tr, pred_te, y_tr)

In [ ]:
# Creates dataframe about grid point's performance
df_pred = create_df_pred(grid).sort_values('mean_test_score', ascending=False)
df_pred.head()

In [ ]:
# Plot prediction behaviour
plot_hyperparam_fitting(df_pred)

## Observation 2

Hmm... things seem to have gotten worse. It's at that time that I've explored multiple other models. SVR (linear/rbf), lasso, elastinet, SGD, KNN, RandomForests, Neural networks, etc. Nothing seems to have help.

# 4. Manual data scaling

Going against my initial intuition, I've tried the scaling of the FNC features. I was curious why some people scaled with a factor of 500, others with 400, 600 or 300. My solution to this conundrum was, let's triangulate the optimal scaling.

The task was straight forward and very iterative. Let's try different values within the variable...

```python
scale_values = [1,        # IC features
                1,        # FNC features
                1,        # intra features
                1]        # inter features

```

and see if this improves the modeling. As much as I can observe, using different scales means also moving the optimal "alpha peak" around. It's kind of like a summazion of different waves where you try to stack the peaks. Changing the scales, changes the peaks location (not necessarily the hight) on the x axis. Problem here is clearly, everytime you add another feature set, the regularization term might shift heavily.

Nonetheless, this path proved to be fruitful. So let's see the effect. Again, finding the right values, was a triangulation/iterative approach.

But before we start, let's package this all into a function.

In [ ]:
def run_prediction(model_metric, estimator=None, alphas=[0.1, 1, 10], cv=5,
                   scaler_targets=None, target='age', tidx=0, scale_values=None):
    
    # Extract dataset
    X_tr, X_te, y_tr = load_dataset_and_scale(target, scale_values=scale_values)
    
    # Create grid search object
    grid = create_grid(model_metric, alphas=alphas, estimator=estimator,
                       cv=cv, scaler_targets=scaler_targets, tidx=tidx)
    
    # Run grid search
    _ = grid.fit(X_tr, y_tr)
    
    # Provide some insights into the models top performance
    print('Dataset scales used: ', scale_values)
    print("Best score at: %f using %s" % (grid.best_score_, grid.best_params_))

    # Extract the predictions for the training and the test set
    pred_tr, pred_te = extract_predictions(X_tr, X_te, grid, y_tr)

    # Plot the predictions with respect to each other and to the target
    plot_predictions(pred_tr, pred_te, y_tr)

    # Creates dataframe about grid point's performance
    df_pred = create_df_pred(grid)
    display(df_pred.sort_values('mean_test_score', ascending=False).head())

    # Plot prediction behaviour
    plot_hyperparam_fitting(df_pred)
    
    return df_pred, pred_tr, pred_te, grid, y_tr

## 4.1. Data modeling with 2 manually scaled feature sets

In [ ]:
# Select the target (name and index)
target = 'age'
tidx = 0

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.25,     # Feature: IC
                0.04,     # Feature: FNC
                np.nan,   # Feature: Intra Corr
                np.nan,   # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = Ridge(tol=1e-3)
alphas = np.logspace(-2, 4, 31)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=alphas, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

## Observation 3

As we saw in other scaling approaches (e.g. scaling with a factor of 500), we can see that the results improved and are now better than ones with the `RobustScaler()`. Let's see what the other two feature sets bring to the table.

## 4.2. Data modeling with 3 manually scaled feature sets

In [ ]:
# Select the target (name and index)
target = 'age'
tidx = 0

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.25,     # Feature: IC
                0.04,     # Feature: FNC
                0.087,    # Feature: Intra Corr
                np.nan,   # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = Ridge(tol=1e-3)
alphas = np.logspace(-2, 4, 31)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=alphas, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

## 4.3. Data modeling with 4 manually scaled feature sets

In [ ]:
# Select the target (name and index)
target = 'age'
tidx = 0

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.25,     # Feature: IC
                0.04,     # Feature: FNC
                0.087,    # Feature: Intra Corr
                0.025,    # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = Ridge(tol=1e-3)
alphas = np.logspace(-2, 4, 31)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=alphas, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

## Observartion 4

As we can see, using these 4 feature datasets with manually scaled properties, we can reduce the mean absolute error for age quiet a bit. However, using a standard `RobustScaler` approach, this advantage gets lost and we are worse off than before.

# 5. Compute predictions for all 5 targets using Ridge

In [ ]:
# Collect test predictions
predictions_ridge = {}

# Save predictions for age in output variable
predictions_ridge[target] = pred_te

## 5.1. Predicting `domain1_var1` using Ridge

In [ ]:
# Select the target (name and index)
target = 'domain1_var1'
tidx = 1

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.25,    # Feature: IC
                0.01,    # Feature: FNC
                0.032,   # Feature: Intra Corr
                0.019,   # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = Ridge(tol=1e-3)
alphas = np.logspace(-1, 5, 31)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=alphas, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

In [ ]:
# Save predictions for domain1_var1 in output variable
predictions_ridge[target] = pred_te

## 5.2. Predicting `domain1_var2` using Ridge

In [ ]:
# Select the target (name and index)
target = 'domain1_var2'
tidx = 2

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.121,   # Feature: IC
                0.019,   # Feature: FNC
                0.032,   # Feature: Intra Corr
                0.025,   # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = Ridge(tol=1e-3)
alphas = np.logspace(-1, 5, 31)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=alphas, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

In [ ]:
# Save predictions for domain1_var2 in output variable
predictions_ridge[target] = pred_te

## 5.3. Predicting `domain2_var1` using Ridge

In [ ]:
# Select the target (name and index)
target = 'domain2_var1'
tidx = 3

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.25,    # Feature: IC
                0.008,   # Feature: FNC
                0.012,   # Feature: Intra Corr
                0.012,   # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = Ridge(tol=1e-3)
alphas = np.logspace(-1, 5, 31)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=alphas, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

In [ ]:
# Save predictions for domain2_var1 in output variable
predictions_ridge[target] = pred_te

## 5.4. Predicting `domain2_var2` using Ridge

In [ ]:
# Select the target (name and index)
target = 'domain2_var2'
tidx = 4

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.261,   # Feature: IC
                0.025,   # Feature: FNC
                0.052,   # Feature: Intra Corr
                0.022,   # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = Ridge(tol=1e-3)
alphas = np.logspace(-1, 5, 31)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=alphas, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

In [ ]:
# Save predictions for domain2_var2 in output variable
predictions_ridge[target] = pred_te

## Conclusion 1

Using the manual scaling approach, together with the four feature datastes is very powerful, but mostly only helps with the prediction of `age`.

It is possible to increase the scores slightly, by focusing in on the grid space and optimize the alpha value (as well as the manual scaling factor) more precisely, but the risk is clearly also there for overfitting.

Increasing the number of folds (i.e. `cv`) helps with some targets and worsens other. But the generalization probably improves nonetheless.

# 6. Compute prediction for all 5 targets using SVR (rbf)

As other's have mentioned in the discussions, Ridge and SVR seem to be very suited to predict the target values. For this reason, let's do the same thing as we did above, but this time with an SVR estimator, using an rbf kernel.

The only thing we need to adapt to make this work is the following three functions.

**Note**: To reduce computation time, we will remove the `RobustScaler` from the grid search.

In [ ]:
def create_grid(model_metric, alphas=[0.1, 1, 10], estimator=None,
                cv=5, scaler_targets=None, tidx=0):

    # Create Pipeline
    pipeline = Pipeline([
        ('estimator', estimator),
    ])

    # Define parameter grid
    param_grid = [{'estimator__C': alphas}]

    # Create grid search object
    f_scorer = make_scorer(model_metric, greater_is_better=False,
                           scaler=scaler_targets, tidx=tidx)
    grid = GridSearchCV(pipeline,
                        cv=cv,
                        param_grid=param_grid,
                        scoring=f_scorer,
                        return_train_score=True,
                        verbose=5,
                        n_jobs=-1)

    return grid

In [ ]:
def create_df_pred(grid):

    # Store grid search parameters and outcomes in dataframe
    df_pred = pd.DataFrame(grid.cv_results_)
    columns = [c for c in df_pred.columns if 'time' not in c
               and 'split' not in c
               and 'rank' not in c
               and c!='params']
    df_pred = df_pred[columns].sort_values('mean_test_score', ascending=False)
    df_pred['param_estimator__C'] = df_pred['param_estimator__C'].astype('float')

    return df_pred

In [ ]:
def plot_hyperparam_fitting(df_pred):

    # Plot the model fit information
    df_plot = df_pred.copy()

    df_plot = df_plot.sort_values('param_estimator__C')

    # Extract relevant modelling metrics
    train_scores = df_plot['mean_train_score']
    valid_scores = df_plot['mean_test_score']
    std_tr = df_plot['std_train_score']
    std_va = df_plot['std_test_score']

    plt.figure(figsize=(12, 4))
    Cs = df_plot['param_estimator__C']
    plt.semilogx(Cs, train_scores, label='Training Set')
    plt.semilogx(Cs, valid_scores, label='Validation Set')

    # Add marker and text for best score
    max_id = np.argmax(valid_scores)
    x_pos = Cs.iloc[max_id]
    y_pos = valid_scores.iloc[max_id]
    txt = '{:0.4f}'.format(y_pos)
    plt.scatter(x_pos, y_pos, marker='x', c='red', zorder=10)
    plt.text(x_pos, y_pos, txt, fontdict={'size': 18})

    # Quantify variance with ±std curves
    plt.fill_between(Cs, train_scores-std_tr, train_scores+std_tr, alpha=0.3)
    plt.fill_between(Cs, valid_scores-std_va, valid_scores+std_va, alpha=0.3)
    plt.ylabel('Performance metric')
    plt.xlabel('Model parameter')

    # Adjust x-lim, y-lim, add legend and adjust layout
    plt.legend()
    plt.show()

## 6.1. Predicting `age` using SVR(rbf)

In [ ]:
# Select the target (name and index)
target = 'age'
tidx = 0

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.25,      # Feature: IC
                0.015,     # Feature: FNC
                0.042,     # Feature: Intra Corr
                0.014,     # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = SVR(gamma='scale', epsilon=0.2, tol=1e-3, max_iter=5000)
Cs = np.logspace(-1, 1, 11)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=Cs, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

In [ ]:
# Collect test predictions
predictions_svr = {}

# Save predictions for age in output variable
predictions_svr[target] = pred_te

## 6.2. Predicting `domain1_var1` using SVR(rbf)

In [ ]:
# Select the target (name and index)
target = 'domain1_var1'
tidx = 1

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.25,      # Feature: IC
                0.012,     # Feature: FNC
                0.032,     # Feature: Intra Corr
                0.018,     # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = SVR(gamma='scale', epsilon=0.2, tol=1e-3, max_iter=5000)
Cs = np.logspace(-1, 1, 11)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=Cs, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

In [ ]:
# Save predictions for age in output variable
predictions_svr[target] = pred_te

## 6.3. Predicting `domain1_var2` using SVR(rbf)

In [ ]:
# Select the target (name and index)
target = 'domain1_var2'
tidx = 2

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.18,      # Feature: IC
                0.01,      # Feature: FNC
                np.nan,    # Feature: Intra Corr (no improvement)
                0.025,     # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = SVR(gamma='scale', epsilon=0.2, tol=1e-3, max_iter=5000)
Cs = np.logspace(-1.5, 0.5, 11)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=Cs, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

In [ ]:
# Save predictions for age in output variable
predictions_svr[target] = pred_te

## 6.4. Predicting `domain2_var1` using SVR(rbf)

In [ ]:
# Select the target (name and index)
target = 'domain2_var1'
tidx = 3

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.25,      # Feature: IC
                0.025,     # Feature: FNC
                0.036,     # Feature: Intra Corr (no improvement)
                0.023,     # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = SVR(gamma='scale', epsilon=0.2, tol=1e-3, max_iter=5000)
Cs = np.logspace(-1, 1, 11)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=Cs, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

In [ ]:
# Save predictions for age in output variable
predictions_svr[target] = pred_te

## 6.5. Predicting `domain2_var2` using SVR(rbf)

In [ ]:
# Select the target (name and index)
target = 'domain2_var2'
tidx = 4

In [ ]:
# Let's select which features to use and how to scale them
scale_values = [0.189,      # Feature: IC
                0.025,      # Feature: FNC
                0.050,    # Feature: Intra Corr (no improvement)
                0.022,     # Feature: Inter Corr
               ]

In [ ]:
# Define model parameters
estimator = SVR(gamma='scale', epsilon=0.2, tol=1e-3, max_iter=5000)
Cs = np.logspace(-1, 1, 11)
cv = 5

In [ ]:
df_pred, pred_tr, pred_te, grid, y_tr = run_prediction(
    model_metric, estimator=estimator, alphas=Cs, cv=cv,
    scaler_targets=scaler_targets, target=target, tidx=tidx,
    scale_values=scale_values)

In [ ]:
# Save predictions for age in output variable
predictions_svr[target] = pred_te

# 7. Saving predictions

In [ ]:
# Load sample submission file
submission = pd.read_csv(opj('/kaggle', 'input', 'trends-assessment-prediction', 'sample_submission.csv')).set_index('Id')
submission.head()

Let's write a function that inverts the initial target feature adapataions (i.e. scaling and power transformation).

In [ ]:
def back_transform(y_test, unique_values, scaler=None, tidx=0):
    
    # List of power transformations
    pow_age = 1.0
    pow_d1v1 = 1.5
    pow_d1v2 = 1.5
    pow_d2v1 = 1.5
    pow_d2v2 = 1.5
    pow_d21 = 1.5
    pow_d22 = 1.0

    powers = [pow_age, pow_d1v1, pow_d1v2, pow_d2v1, pow_d2v2, pow_d21, pow_d22]
    
    # Assign closest value from training set
    for i, a in enumerate(y_test):
        y_test[i] = unique_values[np.argmin(np.abs(a-unique_values))]
    
    # Invert scaler
    y_test = scaler.inverse_transform(np.transpose([y_test] * 7))[:, tidx]
    
    # Invert power transformation
    y_test = np.power(y_test, 1./powers[tidx])

    return y_test

In [ ]:
# Fill up the submission file with the ridge predictions
prediction_dict = predictions_ridge
for i, t in enumerate(['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']):
    unique_values = sorted(targets[t].dropna().unique())
    pred_values = back_transform(prediction_dict[t], unique_values, scaler=scaler_targets, tidx=i)
    submission.iloc[submission.index.str.contains(t), 0] = pred_values

# Let's visualize a few points from the submission file
display(submission.head(10))

# Store predictions in CSF file
submission.to_csv('submission_ridge.csv')

### Score for pure Ridge model

Only using the predictions from the Ridge model, we would have reached a score of 0.15808, which in the final rating would have lead to the 24th place in the public leaderboard.

**Note**: Out of interest, I also ran the predcition with the "feature offset correction" described by the 1st place team (see [here](https://www.kaggle.com/c/trends-assessment-prediction/discussion/163017), which moved the score of the Ridge model to 0.15787, corresponding to the 21st position on the public leaderboard.

In [ ]:
# Fill up the submission file with the ridge predictions
prediction_dict = predictions_svr
for i, t in enumerate(['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']):
    unique_values = sorted(targets[t].dropna().unique())
    pred_values = back_transform(prediction_dict[t], unique_values, scaler=scaler_targets, tidx=i)
    submission.iloc[submission.index.str.contains(t), 0] = pred_values

# Let's visualize a few points from the submission file
display(submission.head(10))

# Store predictions in CSF file
submission.to_csv('submission_svr.csv')

### Score for pure SVR model

Only using the predictions from the SVR model, we would have reached a score of 0.15769, which in the final rating would have lead to the 19th place in the public leaderboard.


**Note**: Out of interest, I also ran the predcition with the "feature offset correction" described by the 1st place team (see [here](https://www.kaggle.com/c/trends-assessment-prediction/discussion/163017), which moved the score of the SVR model to 0.15738, corresponding to the 13th position on the public leaderboard.

# 8. Visualize differences in prediction

As a final last step, let's also look at the prediction differences between the two models.

In [ ]:
for c in predictions_ridge.keys():
    plt.figure(figsize=(6, 6))
    plt.scatter(predictions_ridge[c], predictions_svr[c], s=2, alpha=0.5)
    plt.title(c)
    plt.xlabel('Ridge Prediction')
    plt.ylabel('SVR Prediction')
    plt.show()